<a href="https://colab.research.google.com/github/MDRafiqul-Islam/Large-Scale-Data-Management/blob/main/Tensor_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np

In [ ]:
def eye(n):
  lst = [[] for _ in range(n)]
  for i in range(n):
    for j in range(n):
      if i == j:
        lst[i].append(1)
      else:
        lst[i].append(0)
  return np.array(lst)

In [ ]:
def qr_algorithm(A):
  n = A.shape[0]
  Q = eye(n)

  for i in range(n - 1):
    R = np.zeros((n, n))
    Q_i = np.zeros((n, n))

    for j in range(n):
      R[j, j] = np.sqrt(np.sum(A[:, j] ** 2))
      Q_i[:, j] = A[:, j] / R[j, j]
      R[j, i] = np.sum(Q_i[:, j] * A[:, i])
      A[:, i] = A[:, i] - R[j, i] * Q_i[:, j]
    
    A = R @ Q_i
    Q = Q @ Q_i
  
  eigenvalues = np.diagonal(A)
  eigenvectors = Q
  
  return eigenvalues, eigenvectors

In [ ]:
def svd(A):
    m, n = len(A), len(A[0])
    AtA = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            for k in range(m):
                AtA[i][j] += A[k][i]*A[k][j]
    eigenvalues, eigenvectors = qr_algorithm(AtA)
    singular_values = [math.sqrt(eigenvalue) for eigenvalue in eigenvalues]
    V = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            V[i][j] = eigenvectors[i][j]
    U = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            U[i][j] = A[j][i]/singular_values[i]
    return U, singular_values, V

In [ ]:
def reshape(array, dims):
  if len(array) != np.prod(dims):
    raise ValueError("Invalid dimensions for the reshaped array")
  return np.array(array).reshape(*dims)

In [ ]:
def factor_tensor(T, rank=2):
  # Checking if the rank of the tensor is valid
  if rank > T.ndim or rank < 1:
    raise ValueError("Invalid rank for the tensor")

  # Getting the shape of the tensor
  shape = T.shape

  # Reshapeing the tensor into a matrix
  T_matrix = T.reshape((np.prod(shape[:rank]), np.prod(shape[rank:])))

  U, S, V = svd(T_matrix)

  # Initializeing the two factors with the left and right singular vectors
  T1 = U.reshape(*shape[:rank], -1)
  T2 = V.T.reshape(*shape[rank:], -1)
  return T1, T2

In [ ]:
# Example usage
T = np.random.rand(2, 3, 4)
T1, T2 = factor_tensor(T, rank=2)
print('T = ',T)
print('T1 = ',T1,'\n')
print('T2 = ',T2)

T =  [[[0.55588702 0.63585324 0.41202352 0.39953922]
  [0.59039097 0.88256077 0.78206696 0.64029161]
  [0.52735844 0.74556192 0.44568801 0.08942207]]

 [[0.7066232  0.67765118 0.27694301 0.04729824]
  [0.27661446 0.94609625 0.53567458 0.61643817]
  [0.53381337 0.6102942  0.81643408 0.88447244]]]
T1 =  [[ 1.06083152  5.78013838 12.68149241]
 [ 4.04268768  4.31357604  1.28920275]] 

T2 =  [[ 1.20375332  1.74038611  1.33009801  1.14076433]
 [-0.25240411 -0.35619467 -0.26389684 -0.21717134]
 [ 1.37500906  1.98856051  1.5243148   1.31078107]
 [ 1.2950243   1.87323819  1.43287248  1.23012419]]
